# Write grid shapefile using DIS and BAS

In [24]:
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon
import os
import flopy

In [25]:
model_ws = os.path.join("ReadEnd_WriteCapture","Input","P2R_model")
namefile = 'P2R.nam'

In [26]:
#load MODFLOW model
mf = flopy.modflow.Modflow.load(
    namefile,
    model_ws=model_ws,  # replace with your path
    check=False,
    verbose=False,
    forgive=True,
    version="mf2005",
    load_only=['DIS', 'BAS6']
)

C:\Users\sbramer\AppData\Local\miniconda3\envs\gwm\Lib\site-packages\flopy\mbase.py:97: UserWarning: The program mf2005 does not exist or is not executable.
  warn(


In [27]:
def write_grid_shapefile_from_ibound(ibound, delr, delc, xul, yul, output_file, layer=0):
    """
    Create a shapefile of MODFLOW grid cells from an IBOUND array.

    Parameters:
    - ibound: 3D numpy array (layers, rows, cols)
    - delr: 1D array of column widths
    - delc: 1D array of row heights
    - xul, yul: upper-left model origin (Easting, Northing)
    - output_file: path to output .shp file
    - layer: which layer of IBOUND to use (default: 0)
    """
    nlay, nrow, ncol = ibound.shape
    features = []

    for i in range(nrow):
        for j in range(ncol):
            # Cell edges
            x0 = xul + np.sum(delr[:j])
            x1 = x0 + delr[j]
            y0 = yul - np.sum(delc[:i])
            y1 = y0 - delc[i]

            # Build rectangle polygon
            poly = Polygon([(x0, y0), (x1, y0), (x1, y1), (x0, y1)])

            # Attributes
            row = i + 1
            col = j + 1
            delx = delr[j]
            dely = delc[i]
            area = delx * dely
            ibnd = ibound[layer, i, j]
            rc_str = f"{row}_{col}"

            features.append({
                "geometry": poly,
                "row": row,
                "column": col,
                "delx": delx,
                "dely": dely,
                "area": area,
                "I": row,
                "J": col,
                "IBND": ibnd,
                "R_C": rc_str
            })

    # Build GeoDataFrame and write shapefile
    gdf = gpd.GeoDataFrame(features, crs="EPSG:32149")  # Set to your EPSG
    gdf.to_file(output_file)

    print(f"Shapefile written: {output_file}")

In [28]:
dis = mf.get_package("dis")
delr = mf.dis.delr.array.flatten()
delc = mf.dis.delc.array.flatten()
xul = 557800.0  # easting
yul = 142800.0  # northing
bas = mf.get_package("bas6")
ibound = bas.ibound.array

In [29]:
#write grid shapefile
write_grid_shapefile_from_ibound(
    ibound=ibound,
    delr=delr,
    delc=delc,
    xul=xul,
    yul=yul,
    output_file="P2RSW_grid.shp"
)

Shapefile written: P2RSW_grid.shp
